# Vyhodnotenie zhlukovania a iterovanej dilatácie

**Autor: Bc. Ivan Vykopal**

Pre vyhodnotenie týchto experimentov je potrebné upraviť cesty k skutočným anotáciám od lekárov a k vykonaným predikciám na základe zhlukovania a iterovanej dilatácie.

In [ ]:
import cv2 as cv
import matplotlib as plt
import numpy as np
from shapely.geometry import shape
import glob
import geojson
import pandas as pd
import os

In [ ]:
original_dir = 'D:/Master Thesis/Data/21.06. 2022 annotations/' # path to geojsons from doctors with annotations
predicted_dir = 'D:/Master Thesis/Code/Clustering/result2/' # path to generated geojsons
image_dir = 'D:/Master Thesis/ANN Imunne cells/' # path to full_size images
original_suffix = '.vsi - 20x_annotations.geojson' # suffix for annotations from doctors
predicted_suffix = '.geojson'

csv_name = 'clustering-all-cells4' # name of csv to save

In [ ]:
def get_mask(img, annotations):
    x, y = int(img.shape[0]), int(img.shape[1])

    mask = np.zeros((x, y), dtype='uint8')
    for feat in annotations:
        classification = feat['properties'].get('classification')
        if classification and classification['name'] == 'Inflammation':
            for coors in feat['geometry']['coordinates']:
                if feat['geometry']['type'] == 'MultiPolygon':
                    coors = coors[0]
                pts = [[round(c[0]), round(c[1])] for c in coors]
                cv.fillPoly(mask, [np.array(pts)], 1)  # fill with ones if cell present

    return mask

In [ ]:
def evaluate(original_mask, predicted_mask):
    overlap = original_mask * predicted_mask
    union = original_mask + predicted_mask
    union = np.clip(union, 0, 1)
    
    if overlap.sum() == 0 and union.sum() == 0:
        return 1, 1
    
    IOU = overlap.sum() / float(union.sum())
    
    dice = 2 * (overlap.sum()) / (union.sum() + overlap.sum())
    
    return IOU, dice

In [ ]:
files = glob.glob(f"{original_dir}*")

dataframe = pd.DataFrame()

for file in files:
    file_name = file.replace('\\', '/').replace(original_dir, '').replace(original_suffix, '')
    if os.path.exists(f"{image_dir}{file_name}.png"):
        image = cv.imread(f"{image_dir}{file_name}.png")

        original_geojson = geojson.load(open(file))
        predicted_geojson = geojson.load(open(f"{predicted_dir}{file_name}{predicted_suffix}"))
        original_mask = get_mask(image, original_geojson['features'])
        predicted_mask = get_mask(image, predicted_geojson['features'])
        IoU, dice = evaluate(original_mask, predicted_mask)
        
        dataframe = pd.concat([
                dataframe,
                pd.DataFrame([{
                    'file_name': file_name,
                    'IoU': IoU,
                    'Dice': dice
                }]).set_index('file_name')
            ])
    

In [ ]:
dataframe

In [ ]:
dataframe.to_csv(f'{csv_name}.csv')